In [4]:
# Pinecone 적용과 RAG 파이프라인 구축

from dotenv import load_dotenv
import os
from langchain_community.vectorstores import Pinecone as LangchainPinecone
from langchain_upstage import UpstageDocumentParseLoader, UpstageEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_upstage import ChatUpstage
from langchain.chains import LLMChain
from langchain.prompts.chat import ChatPromptTemplate
from langchain.output_parsers import RetryOutputParser
from pinecone import Pinecone, ServerlessSpec

# 1. 환경 변수 로드
load_dotenv()



c:\Users\SSAFY\AppData\Local\Programs\Python\Python312\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


True

In [9]:
# Pinecone 클라이언트 설정
pc = Pinecone(
    api_key=os.getenv("PINECONE_API_KEY"),
)

In [27]:
# Pinecone 인덱스 생성
index_name = "multilingual-e5-large"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=4096,  # 임베딩 벡터 크기
        metric="euclidean",
        spec=ServerlessSpec(
            cloud="aws",
            region=os.getenv("PINECONE_ENVIRONMENT"),
        ),
    )

In [26]:
# pc.delete_index(index_name)
## 임베팅 모델 출력 차원에 맞게 설정할려고 지움.
## 임베딩 모델의 출력 차원이 4096, 따라서, 처음에 설정했던 1024에서 4096로 바꿈꿈

In [15]:
# 2. 데이터 수집 및 문서 로드
directory_path = r"C:\Users\SSAFY\Desktop\AI\data"
file_paths = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if file.endswith('.pdf')]

print(f"Files to process: {file_paths}")

# 문서 로드
loaded_documents = []
for file_path in file_paths:
    try:
        loader = UpstageDocumentParseLoader(file_path, 
                                            output_format='html', 
                                            coordinates=False)
        document = loader.load()
        
        # 파일명 metatdata에 추가
        for page in document:
            page.metadata["file_name"] = os.path.basename(file_path)  
        loaded_documents.append(document)
        print(f"Loaded document: {file_path}")
    except Exception as e:
        print(f"Error loading {file_path}: {e}")

Files to process: ['C:\\Users\\SSAFY\\Desktop\\AI\\data\\국민은행_신년사.pdf', 'C:\\Users\\SSAFY\\Desktop\\AI\\data\\한국은행_신년사.pdf']
Loaded document: C:\Users\SSAFY\Desktop\AI\data\국민은행_신년사.pdf
Loaded document: C:\Users\SSAFY\Desktop\AI\data\한국은행_신년사.pdf


In [16]:
# 3. 청크 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
texts = []
for doc in loaded_documents:
    texts.extend(text_splitter.split_documents(doc))

print(f"전체 청크 개수: {len(texts)}")
print(texts[1].metadata)

전체 청크 개수: 11
{'total_pages': 4, 'file_name': '국민은행_신년사.pdf'}


In [28]:
# 4. 임베딩 생성 및 벡터 DB 구축
embeddings = UpstageEmbeddings(model="solar-embedding-1-large")

# 텍스트와 메타데이터, ID 매핑
texts_to_store = [text.page_content for text in texts]
metadatas = [text.metadata for text in texts]
ids = [f"text-{i}" for i in range(len(texts))]  # 고유 ID 생성

# 벡터 DB 생성 및 데이터 추가
vectorstore = LangchainPinecone.from_texts(
    texts=texts_to_store,
    embedding=embeddings,
    metadatas=metadatas,
    ids=ids,
    index_name=index_name
)

================================================================================
### App.py 부분

In [2]:
from dotenv import load_dotenv
import os
from langchain_community.vectorstores import Pinecone as LangchainPinecone
from langchain_upstage import UpstageDocumentParseLoader, UpstageEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_upstage import ChatUpstage
from langchain.chains import LLMChain
from langchain.prompts.chat import ChatPromptTemplate
from langchain.output_parsers import RetryOutputParser
from pinecone import Pinecone, ServerlessSpec

c:\Users\SSAFY\AppData\Local\Programs\Python\Python312\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
# 1. 환경 변수 로드
load_dotenv()

# Pinecone 클라이언트 설정
pc = Pinecone(
    api_key=os.getenv("PINECONE_API_KEY"),
)

In [6]:
# Pinecone 검색 : 이미 생성한 Pinecone 인덱스를 불러오기

vectorstore = LangchainPinecone.from_existing_index(
    index_name='ssafy-index',  # 기존에 사용한 인덱스 이름
    embedding=UpstageEmbeddings(model="solar-embedding-1-large")  # 동일한 임베딩 모델
)

In [7]:
# 5. Retriever 설정 : MMR(Minimal Marginal Relevance) 알고리즘을 사용하여 관련성 높은 문서 검색
retriever = vectorstore.as_retriever(
    search_type="mmr",  # mmr 알고리즘으로 검색
    search_kwargs={"k": 3, "threshold": 0.7}  # 검색 결과 상위 3개 반환, 유사도 임계값 0.7
)

In [15]:

#6. 프롬프트 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", """
        너는 기업 신년사 PDF 문서로 학습된 기업 분석 AI 챗봇입니다. 사용자 질문에 대해 문서에 있는 내용으로로 답변하세요.
        문서에 없는 내용이라면 '알 수 없습니다'라고 답변하세요.
        아래는 출력 형식 예제입니다. 항상 동일한 형식으로 출력해야 합니다.
        주요 키워드는 3개 이하, 지원 관련 경험 추천도 3개 이하입니다.
        
        ### 출력 형식 예제
        | **구분**            | **내용**                                                                                 |
        |---------------------|-----------------------------------------------------------------------------------------|
        | **정보 요약**       | LG의 신년사에서는 고객 중심 경영 강화, 신사업 영역 확대(AI 및 친환경 기술), 내부 디지털 전환 가속화가 주요 전략으로 제시되었습니다. |
        | **주요 키워드**     | 고객 중심, 신사업(AI, 친환경), 디지털 전환, 글로벌 시장 확대                      |
        | **지원 관련 경험 추천** | - 고객 중심 경영: 과거 고객 데이터를 분석하여 서비스 개선 경험<br>- 신사업 영역: AI 기반 프로젝트에 참여하여 새로운 서비스 설계 경험<br>- 디지털 전환: 기업 내 시스템 디지털화 프로젝트 수행 경험 |

        문서가 없는 경우의 출력 예제:
        | **구분**            | **내용**                     |
        |---------------------|-----------------------------|
        | **정보 요약**       | 문서에 해당 질문에 대한 정보가 없습니다. |
        | **주요 키워드**     | 없음                        |
        | **지원 관련 경험 추천** | 없음                        |

        항상 위와 같은 형식을 유지하세요.
        ---
        CONTEXT:
        {context}
    """),
    ("human", "{input}")
])

# LLM 모델 설정
llm = ChatUpstage(model="solar-pro")

# 체인 구성
chain = LLMChain(
    prompt=prompt,
    llm=llm)  # output_parser=RetryOutputParser() 안됨됨

In [1]:
import time
from concurrent.futures import ThreadPoolExecutor, TimeoutError

# 7. 질문 및 답변 생성
query = "국민은행의 주요 사업 전략은 무엇인가요?"

# 7-1. 문서 검색
start_time = time.time()
retrieved_docs = retriever.invoke(query)
print(f"Retriever 실행 시간: {time.time() - start_time:.2f}초")

# 7-2. 검색된 문서 텍스트 가져오기
context = "\n".join([doc.page_content[:500] for doc in retrieved_docs])

# 7-3. 응답 생성 (타임아웃 설정)
def generate_response():
    return chain.invoke({"context": context, "input": query})

try:
    with ThreadPoolExecutor() as executor:
        future = executor.submit(generate_response)
        response = future.result(timeout=30)  # 30초로 제한
        print("Generated Response:", response)
except TimeoutError:
    print("응답 생성 시간이 초과되었습니다. 작업이 강제 종료됩니다.")

NameError: name 'retriever' is not defined

In [44]:
len(retrieved_docs)

4